In [1]:
import torch
from pathlib import Path
import json

from adaptive_al_v2.active_learning import ActiveLearning, ExperimentConfig

import logging
# Comment it out if you dont want to see info logs
logging.basicConfig(level=logging.INFO)

C:\Users\saba\miniconda3\envs\ds-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [3]:
cfg = ExperimentConfig(
    seed=42,
    total_rounds=5,
    experiment_name="dummy_test_pipeline",
    save_dir=Path("./experiments"),

    # Pool settings
    initial_pool_size=200,
    acquisition_batch_size=256,

    # Model
    model_name_or_path="distilbert-base-uncased",
    num_labels=4, # TODO: maybe make it figure it out on its own based on dataset
    tokenizer_kwargs={
        "max_length": 128,
        "padding": "max_length",
        "truncation": True,
        "add_special_tokens": True,
        "return_tensors": "pt"
    },

    # Dataset names (for reference)
    data="agnews",

    # Strategy
    strategy_class="DeltaF1Strategy",
    strategy_kwargs={"epsilon": 0.01, "k": 2}, # The base params are passed internally, only strategy specific params needed here

    optimizer_class = "Adam",
    optimizer_kwargs = {"lr": 1e-3, "weight_decay": 1e-4},

    criterion_class = "CrossEntropyLoss",
    criterion_kwargs = {},

    scheduler_class = "StepLR",
    scheduler_kwargs = {"step_size": 10, "gamma": 0.1},

    # Sampler
    sampler_class="RandomSampler",
    sampler_kwargs={"seed": 42},
    # sampler_class="EntropySampler",
    # sampler_kwargs={"show_progress": True},

    # Training
    device=device,
    epochs=3,
    batch_size=64
)

In [4]:
al = ActiveLearning(cfg)

INFO:root:Loading tokenizer and model from 'distilbert-base-uncased'...
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:root:Train size: 108000, Validation size: 12000, Test size: 7600


## Just checking if it actually works

In [5]:
print(f"Initial pool stats: {al.pool.get_pool_stats()}")

Initial pool stats: {'labeled_count': 100, 'unlabeled_count': 107900, 'total_count': 108000}


In [6]:
round_stats = al.train_one_round(new_indices=None)
print(f"Round 1 completed. Val F1: {round_stats['f1_score']:.4f}, Training Time: {round_stats['training_time']:.2f}s")

INFO:root:
--- Round 1
INFO:root:Resetting model to initial state . . .
INFO:root:delta_f1: 0, 1 consecutive rounds.
INFO:root:Round 1 complete. Val Stats: Loss=1.2112274670854528, F1=0.36714620256896324, Time=61.96s


Round 1 completed. Val F1: 0.3671, Training Time: 61.96s


In [7]:
new_indices = al.sample_next_batch()
print(f"Sampled {len(new_indices)} new indices: {new_indices[:5]} ...")

INFO:root:Sampled 256 new samples using RandomSampler


Sampled 256 new indices: [83881, 14608, 3281, 97279, 36077] ...


In [8]:
round_stats = al.train_one_round(new_indices=new_indices)
print(f"Round 2 completed. Val F1: {round_stats['f1_score']:.4f}, Training Time: {round_stats['training_time']:.2f}s")

INFO:root:
--- Round 2
INFO:root:Training with 256 new samples
INFO:root:Resetting model to initial state . . .
INFO:root:delta_f1: 0.22469853091203285, 0 consecutive rounds.
INFO:root:Round 2 complete. Val Stats: Loss=0.81671791365172, F1=0.5918447334809961, Time=66.77s


KeyError: (56133, 101996, 103133, 27327, 17750)

In [13]:
num_additional_rounds = 3
for r in range(num_additional_rounds):
    print(f"\n--- Round {al.current_round + 1}")

    new_indices = al.sample_next_batch()
    if not new_indices:
        print("No more unlabeled data available!")
        break

    round_stats = al.train_one_round(new_indices=new_indices)
    print(f"Val F1: {round_stats['f1_score']:.4f}, Training Time: {round_stats['training_time']:.2f}s")
    print(f"Pool Stats: {round_stats['pool_stats']}")

INFO:root:Sampled 256 new samples using RandomSampler
INFO:root:
--- Round 6
INFO:root:Training with 256 new samples



--- Round 6


INFO:root:Round 6 complete. Val Stats: Loss=1.325510044047173, F1=0.19949427609107603, Time=40.00s
INFO:root:Sampled 256 new samples using RandomSampler
INFO:root:
--- Round 7
INFO:root:Training with 256 new samples


Val F1: 0.1995, Training Time: 40.00s
Pool Stats: {'labeled_count': 1380, 'unlabeled_count': 106620, 'total_count': 108000}

--- Round 7


INFO:root:Round 7 complete. Val Stats: Loss=1.2804185843214075, F1=0.24051267661223574, Time=45.39s
INFO:root:Sampled 256 new samples using RandomSampler
INFO:root:
--- Round 8
INFO:root:Training with 256 new samples


Val F1: 0.2405, Training Time: 45.39s
Pool Stats: {'labeled_count': 1636, 'unlabeled_count': 106364, 'total_count': 108000}

--- Round 8


INFO:root:Round 8 complete. Val Stats: Loss=1.2802721496592178, F1=0.24539144035730043, Time=53.01s


Val F1: 0.2454, Training Time: 53.01s
Pool Stats: {'labeled_count': 1892, 'unlabeled_count': 106108, 'total_count': 108000}


# FULL PIPELINE HERE ! ! !

In [14]:
final_metrics = al.run_full_pipeline()
print(f"Final Test Metrics: F1={final_metrics['f1_score']:.4f}, Accuracy={final_metrics['accuracy']:.4f}, Loss={final_metrics['loss']:.4f}")

INFO:root:Loading tokenizer and model from 'distilbert-base-uncased'...
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:root:Train size: 108000, Validation size: 12000, Test size: 7600
INFO:root:Running 5 rounds.
INFO:root:
--- Round 1
INFO:root:Round 1 complete. Val Stats: Loss=1.389565756980409, F1=0.10637014967723629, Time=2.88s
INFO:root:Sampled 256 new samples using RandomSampler
INFO:root:
--- Round 2
INFO:root:Training with 256 new samples
INFO:root:Round 2 complete. Val Stats: Loss=1.3485149608013478, F1=0.13607678299913153, Time=10.12s
INFO:root:Sampled 256 new samples using RandomSampler
INFO:root:
--- Round 3
INFO:root:Training with 256 new samples
INFO:root:Round 3 comp

Final Test Metrics: F1=0.3048, Accuracy=0.4221, Loss=1.2131


In [23]:
al.save_experiment()

INFO:root:Experiment saved to experiments\dummy_test_pipeline\results_20250829_171203.json


In [31]:
with open(r"experiments/dummy_test_pipeline/results_20250829_171203.json", 'r') as f:
    experiment_data = json.load(f)

In [32]:
print(experiment_data.keys())

dict_keys(['cfg', 'total_rounds', 'round_val_stats', 'final_pool_stats', 'final_test_stats'])


In [33]:
experiment_data['cfg']

{'seed': 42,
 'total_rounds': 5,
 'initial_pool_size': 100,
 'acquisition_batch_size': 256,
 'sampler_class': 'RandomSampler',
 'sampler_kwargs': {},
 'strategy_class': 'FineTuneStrategy',
 'strategy_kwargs': {},
 'model_name_or_path': 'distilbert-base-uncased',
 'num_labels': 4,
 'tokenizer_kwargs': {'max_length': 128,
  'padding': 'max_length',
  'truncation': True,
  'add_special_tokens': True,
  'return_tensors': 'pt'},
 'optimizer_class': 'Adam',
 'optimizer_kwargs': {'lr': 0.001, 'weight_decay': 0.0001},
 'criterion_class': 'CrossEntropyLoss',
 'criterion_kwargs': {},
 'scheduler_class': 'StepLR',
 'scheduler_kwargs': {'step_size': 10, 'gamma': 0.1},
 'device': 'cuda',
 'epochs': 3,
 'batch_size': 64,
 'data': 'agnews',
 'save_dir': 'experiments',
 'experiment_name': 'dummy_test_pipeline'}

In [34]:
experiment_data['total_rounds']

5

In [35]:
experiment_data['round_val_stats'][-1]

{'training_time': 31.023064613342285,
 'avg_loss': 1.2513243467719466,
 'epochs': 3,
 'total_samples': 1124,
 'new_samples': 256,
 'loss': 1.2030607924816457,
 'f1_score': 0.3041531736197598,
 'accuracy': 0.42633333333333334,
 'pool_stats': {'labeled_count': 1124,
  'unlabeled_count': 106876,
  'total_count': 108000}}

In [36]:
experiment_data["final_pool_stats"]

{'labeled_count': 1124, 'unlabeled_count': 106876, 'total_count': 108000}

In [37]:
experiment_data["final_test_stats"]

{'loss': 1.213091655939567,
 'f1_score': 0.3047702822865449,
 'accuracy': 0.42210526315789476}